# Data Exploration

This notebook explores the datasets to understand:
- Data distributions
- Missing values
- Relationships between variables
- Historical yield patterns


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set up paths
data_dir = Path.cwd() / 'raw_datasets'
if not data_dir.exists():
    data_dir = Path('../raw_datasets').resolve()
output_dir = Path("/tmp/models")
output_dir.mkdir(exist_ok=True)

# Load datasets
print("Loading datasets...")
climate_df = pd.read_csv(data_dir / "climate_data.csv")
crop_climate_req = pd.read_csv(data_dir / "crop_climate_requirements.csv")
crop_requirements = pd.read_csv(data_dir / "crop_requirements.csv")
crop_npk_req = pd.read_csv(data_dir / "crop_npk_level_requirements.csv")
historical_perf = pd.read_csv(data_dir / "historical_crop_performance.csv")

print(f"Climate data: {len(climate_df)} records")
print(f"Crop climate requirements: {len(crop_climate_req)} crops")
print(f"Crop requirements: {len(crop_requirements)} crops")
print(f"NPK requirements: {len(crop_npk_req)} crops")
print(f"Historical performance: {len(historical_perf)} records")


In [ ]:
# Explore historical performance data
print("Historical Performance Data Overview:")
print(historical_perf.head())
print("\nColumns:", historical_perf.columns.tolist())
print("\nData types:")
print(historical_perf.dtypes)
print("\nMissing values:")
print(historical_perf.isnull().sum())
print("\nBasic statistics:")
print(historical_perf.describe())


In [ ]:
# Calculate yield per hectare
historical_perf['yield_per_ha'] = (
    historical_perf['Volume_Production'] / historical_perf['Area_Planted_Harvested']
)

# Remove invalid yields
historical_perf_clean = historical_perf[
    (historical_perf['yield_per_ha'].notna()) & 
    (historical_perf['yield_per_ha'] != float('inf')) &
    (historical_perf['yield_per_ha'] > 0)
].copy()

print(f"Valid yield records: {len(historical_perf_clean)}")
print(f"\nYield statistics (tons/ha):")
print(historical_perf_clean['yield_per_ha'].describe())

# Plot yield distribution
plt.figure(figsize=(10, 6))
plt.hist(historical_perf_clean['yield_per_ha'], bins=50, edgecolor='black')
plt.xlabel('Yield (tons/ha)')
plt.ylabel('Frequency')
plt.title('Distribution of Crop Yields')
plt.show()


## Deeper Dive into Yield Patterns

Let's explore the yield data more deeply to identify patterns and insights.

In [ ]:
# Top 20 Crops by Average Yield
top_crops = historical_perf_clean.groupby('Crop')['yield_per_ha'].mean().sort_values(ascending=False).head(20)

plt.figure(figsize=(12, 8))
top_crops.plot(kind='bar')
plt.title('Top 20 Crops by Average Yield (tons/ha)')
plt.xlabel('Crop')
plt.ylabel('Average Yield (tons/ha)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Top 20 Provinces by Average Yield
top_provinces = historical_perf_clean.groupby('Province')['yield_per_ha'].mean().sort_values(ascending=False).head(20)

plt.figure(figsize=(12, 8))
top_provinces.plot(kind='bar')
plt.title('Top 20 Provinces by Average Yield (tons/ha)')
plt.xlabel('Province')
plt.ylabel('Average Yield (tons/ha)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Yield Trends Over Time
yield_over_time = historical_perf_clean.groupby('Year')['yield_per_ha'].mean()

plt.figure(figsize=(12, 6))
yield_over_time.plot(kind='line', marker='o')
plt.title('Average Crop Yield Over Time')
plt.xlabel('Year')
plt.ylabel('Average Yield (tons/ha)')
plt.grid(True)
plt.show()

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(8, 6))
correlation_matrix = historical_perf_clean[['Volume_Production', 'Area_Planted_Harvested', 'yield_per_ha']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Box Plots for Top 5 Crops
top_5_crops_list = top_crops.index[:5].tolist()
top_crops_df = historical_perf_clean[historical_perf_clean['Crop'].isin(top_5_crops_list)]

plt.figure(figsize=(15, 8))
sns.boxplot(x='Crop', y='yield_per_ha', data=top_crops_df)
plt.title('Yield Distribution for Top 5 Crops')
plt.xlabel('Crop')
plt.ylabel('Yield (tons/ha)')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
historical_perf_clean.tail()